In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [19]:
df = pd.read_csv("../data/train_data_with_genre.csv")
df.head()

,track_id,track_name,artist_id,artist_name,popularity,album_name,genre,artist_genres,duration_ms,explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,analysis_url,time_signature
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,100,Die With A Smile,pop,"art pop,dance pop,pop",251667,False,...,-7.777,0,0.0304,0.30800,0.000000,0.1220,0.535,157.969,https://api.spotify.com/v1/audio-analysis/2plb...,3
1,5R8dQOPq8haW94K7mgERlO,Poker Face,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,76,The Fame,pop,"art pop,dance pop,pop",237200,False,...,-4.618,1,0.0787,0.11900,0.000002,0.1210,0.774,119.001,https://api.spotify.com/v1/audio-analysis/5R8d...,4
2,0SiywuOBRcynK0uKGWdCnn,Bad Romance,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,82,The Fame Monster (Deluxe Edition),pop,"art pop,dance pop,pop",294573,True,...,-3.752,1,0.0363,0.00314,0.000053,0.0842,0.714,119.007,https://api.spotify.com/v1/audio-analysis/0Siy...,4
3,2x7MyWybabEz6Y6wvHuwGE,Just Dance,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,74,The Fame,pop,"art pop,dance pop,pop",241933,False,...,-4.541,0,0.0311,0.02640,0.000043,0.1810,0.745,118.990,https://api.spotify.com/v1/audio-analysis/2x7M...,4
4,11BKm0j4eYoCPPpCONAVwA,Bloody Mary,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,73,Born This Way (Special Edition),pop,"art pop,dance pop,pop",244760,False,...,-6.365,0,0.0291,0.01070,0.000002,0.1130,0.440,99.998,https://api.spotify.com/v1/audio-analysis/11BK...,4


In [20]:
df = df.sample(frac=1)

In [21]:
def bpm_to_tempo(bpm):
    if bpm <= 20:
        return 0  # Larghissimo
    elif 21 <= bpm <= 40:
        return 1  # Grave
    elif 41 <= bpm <= 50:
        return 2  # Lento
    elif 51 <= bpm <= 60:
        return 3  # Largo
    elif 61 <= bpm <= 66:
        return 4  # Larghetto
    elif 67 <= bpm <= 75:
        return 5  # Adagio
    elif 76 <= bpm <= 80:
        return 6  # Adagietto
    elif 81 <= bpm <= 100:
        return 7  # Andante
    elif 101 <= bpm <= 120:
        return 8  # Moderato
    elif 121 <= bpm <= 160:
        return 9  # Allegro
    elif 161 <= bpm <= 175:
        return 10  # Vivace
    elif 176 <= bpm <= 200:
        return 11  # Presto
    else:
        return 12  # Prestissimo

classical_tempo = [bpm_to_tempo(bpm) for bpm in df["tempo"]]
df["tempo"] = classical_tempo

In [22]:
X = df.drop(columns=['track_id', 'track_name', 'artist_id', 'artist_name', 'popularity', 'key', 'mode',
       'album_name', 'genre', 'artist_genres', 'analysis_url', 'duration_ms'])
X.head()

,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
479,False,0.389,0.431,-8.061,0.0344,0.2490,0.0000,0.1300,0.275,9,4
6866,False,0.619,0.966,-4.782,0.1840,0.0694,0.8340,0.8050,0.076,9,4
4475,False,0.404,0.695,-6.162,0.0340,0.5870,0.0383,0.1590,0.646,9,4
4817,False,0.723,0.708,-8.711,0.0478,0.7370,0.0000,0.1420,0.955,9,4
6232,True,0.752,0.910,-3.189,0.0541,0.4810,0.0000,0.0947,0.798,7,4


In [23]:
X_encoded = pd.get_dummies(X, columns=["time_signature"])

In [24]:
continuous_cols = ['danceability', "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence"]
scaler = StandardScaler()
X_encoded[continuous_cols] = scaler.fit_transform(X_encoded[continuous_cols])

In [25]:
y = df["genre"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, stratify=y)

In [27]:
rf = RandomForestClassifier(n_estimators=750)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=750)

In [28]:
rf.score(X_test, y_test)

0.537371763556424

# => Random Forest Accuracy: 0.537